In [ ]:
import mne
%pip install mne_bids
%pip install --upgrade mne_bids

from mne_bids import BIDSPath, print_dir_tree, write_raw_bids, make_dataset_description, update_sidecar_json

In [ ]:
#Allows us to work with file paths
import os.path as op
#Makes file path handling simpler
from pathlib import Path

In [ ]:
#Change the file path to your data's location
data_dir = Path(r"C:\N8_internship_code\second_tester_data\source_data")
print_dir_tree(data_dir, max_depth=1)

In [ ]:
children = [child for child in data_dir.iterdir()]
children

In [ ]:
#Change this to match the file number
dir_number = 0
files = [file for file in children[dir_number].iterdir()]

In [ ]:
file_path = files[0]
file_path

In [ ]:
data = mne.io.read_raw_eeglab(file_path)

In [ ]:
data

In [ ]:
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from mne.transforms import Transform

data.info["device_info"] = {
    "type": "Emotiv Epoc X",
    "model": "12-channel EEG",
    "serial": 33456423,
    "site": "yes"
    }



# Generates the (approx) birthdate of the participant based on the measurement date and age
data.set_meas_date(datetime(2015, 6, 7, tzinfo= timezone.utc))
recording_date = data.info["meas_date"]
Birthdate = recording_date - relativedelta(years=30)


data.info["subject_info"] = {
    "id": 1,
    "his_id": "sub-001",
    "last_name": "Doe",
    "first_name": "John",
    "middle_name": "A",
    "birthday": Birthdate,
    "sex": 2,
    "hand": 1,
    "weight": 70.0,
    "height": 175.0,
}

data.info["line_freq"] = 50

data.info["bads"] = ["C3", "F3" ]

data.info["description"] = "a resting state dataset"

data.info["dev_head_t"] = Transform("meg", "head")

data.info["experimenter"] = "John Doe"

data.info["helium_info"] = {
    "he_level_raw": 20,
    "helium_level": 12,
    "orig_file_guid": "1234567890abcdef",
    "meas_date": (datetime(2015, 6, 7, tzinfo= timezone.utc))
    }


In [ ]:
data

In [ ]:
bids_root = op.join(data_dir.parent, "bids_example")
bids_root

In [ ]:
mne.channels.get_builtin_montages()

In [ ]:
builtin_montages = mne.channels.get_builtin_montages(descriptions=True)
for montage_name, montage_description in builtin_montages:
    print(f"{montage_name}: {montage_description}")

In [ ]:
import mne

my_montage = mne.channels.make_standard_montage("biosemi64")

# Printing montage information
print(my_montage)

# Viualising montage in 2D
my_montage.plot()

# Visualising montage in 3D
fig = my_montage.plot(kind="3d", show=False)  # 3D
fig = fig.gca().view_init(azim=70, elev=15)  # set view angle

In [ ]:
data.set_montage(my_montage, match_case=True, match_alias=False, on_missing='ignore', verbose=None)

In [ ]:
#Edit this information ot match your data
subject_id = "001"
task = "rest"

bids_path = BIDSPath(subject=subject_id, task=task, root=bids_root)
write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EEGLAB")

In [ ]:
#Edit this information ot match your data
subject_id = "001"
task = "rest"

bids_path = BIDSPath(subject=subject_id, task=task, root=bids_root)
write_raw_bids(data, bids_path, overwrite=True, allow_preload=True, format="EEGLAB")

In [ ]:
#creating a dataset description JSON file (separate to other data entry types) [for people who want a better description of their dataset than previously provided] 
# #{will overwrite any existing dataset_description.json file in the root of the BIDS directory}
make_dataset_description(
    path=bids_root,
    name="EEGManyLabs Resting State Study", 
    hed_version="1",
    dataset_type='raw',
    data_license="CCO",
    authors=["Ariana Williams", "Daniel Brady"],
    acknowledgements=None,
    how_to_acknowledge="Cite (Williams et al., 2025) when using this dataset",
    funding=["The NHS", "The Uk government"],
    ethics_approvals="Ethical approval was granted by the University of Leeds School of Psychology Ethics committee (12345 2025)",
    references_and_links="https://mne.tools/mne-bids/stable/whats_new_previous_releases.html",
    doi="doi:https://doi.org/10.1016/j.tins.2017.02.004",
    generated_by=[
        {
            "Name": "MNE-BIDS",
            "Version": "0.14",
            "Description": "Used to convert MEG data into BIDS format."
        },
        {
            "Name": "MNE-Python",
            "Version": "1.6.1",
            "Description": "Used for MEG preprocessing and analysis."
        }
    ],
    source_datasets=[
        {
            "URL": "https://example.com/source_dataset",
            "DOI": "10.1234/example.doi",
        }],
            overwrite=True,
            verbose=True)